In [1]:
from helpers.strings import get_obj_from_path
from helpers.apps.reportings import get_lane
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
companies = Company.objects.filter(name__in=['Arteris Fernão Dias - Conserva'])
companies

<QuerySet [<Company: 31938021-6b32-461b-821c-4759a5016441: Arteris Fernão Dias - Conserva>]>

In [3]:
def get_connected_reference(company, resource, key, value):
    select_options = get_obj_from_path(company.custom_options, "{}__fields__{}__selectoptions__options".format(resource, key))
    return next(a['name'] for a in select_options if a['value'] == value)

In [8]:
unidades=[]
for company in tqdm(companies):
    if company.name not in unidades:
        unidades.append(company.name)
        pd.DataFrame().to_excel(company.name[:14]+'_arvore.xlsx')
    counter=0
    my_dict={}
    for form in tqdm(OccurrenceType.objects.filter(company=company).order_by('occurrence_kind', 'name')):
        
        try:
            campos=[]
            for field in form.form_fields['fields']:
                try:
                    campos.append(field['displayName'])
                except:
                    campos.append(field['display_name'])
            natureza = get_connected_reference(company,
                                                    "reporting",
                                                    "occurrence_kind",
                                                    form.occurrence_kind)
            my_dict[(natureza,form.name)]=campos
            counter+=1
        except StopIteration:
            continue

    max_length = max(len(v) for v in my_dict.values())
    for k, v in my_dict.items():
        my_dict[k] = v + [None] * (max_length - len(v))
    df2 = pd.DataFrame(my_dict)
    df2= df2.transpose()
    with pd.ExcelWriter(company.name[:14]+'_arvore.xlsx', mode='a',if_sheet_exists='replace') as writer:
        df2.to_excel(writer,sheet_name=company.name,na_rep="-")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/138 [00:00<?, ?it/s]

In [9]:
df2

0   \
Levantamento                    Fiscalização                                      PLANEJAMENTO DIÁRIO DE SEGURANÇA DO TRABALHO ...   
Drenagem                        Base de concreto para Pórtico e Semi- Pórtico                                               Origem   
                                Drenagem Profunda                                                                           Origem   
                                Drenagem Superficial                                                                        Origem   
                                Implantação de catadioptrico de barreira rigida                                             Origem   
...                                                                                                                            ...   
Cercas, alambrados e telamentos Reparo (Cercas/Alambrados/Telamento)                                                        Origem   
Defensa Metálica                Alinhamento (Defensa Metálica)                                                              Origem   
                                Implantação (Defensas metálicas)                                                            Origem   
                                Remoção (Defensa Metálica)                                                                  Origem   
                                Reparo (Defensa Metálica)                                                                   Origem   

                                                                                                                                1   \
Levantamento                    Fiscalização                                     OBS -  PLANEJAMENTO DIÁRIO DE SEGURANÇA DO TRA...   
Drenagem                        Base de concreto para Pórtico e Semi- Pórtico                                       Código Interno   
                                Drenagem Profunda                                                                   Código Interno   
                                Drenagem Superficial                                                                Código Interno   
                                Implantação de catadioptrico de barreira rigida                                     Código Interno   
...                                                                                                                            ...   
Cercas, alambrados e telamentos Reparo (Cercas/Alambrados/Telamento)                                                Código Interno   
Defensa Metálica                Alinhamento (Defensa Metálica)                                                      Código Interno   
                                Implantação (Defensas metálicas)                                                    Código Interno   
                                Remoção (Defensa Metálica)                                                          Código Interno   
                                Reparo (Defensa Metálica)                                                           Código Interno   

                                                                                                                             2   \
Levantamento                    Fiscalização                                      TODOS COLABORADORES TEM  INTEGRAÇÃO (CRACHÁS)   
Drenagem                        Base de concreto para Pórtico e Semi- Pórtico                                       Código Kcor   
                                Drenagem Profunda                                                                   Código Kcor   
                                Drenagem Superficial                                                                Código Kcor   
                                Implantação de catadioptrico de barreira rigida                                     Código Kcor   
...                                                                                                                         

In [11]:
OccurrenceType.objects.filter(company=companies[1]).count()

398

In [12]:
companies[1]

<Company: 9763a1c5-3be6-4ddb-90bc-e6b96cc60b8d: Arteris Litoral Sul>

# Lista com os campos

In [4]:
unidades=[]
for company in tqdm(companies):
    if company.name not in unidades:
        unidades.append(company.name)
        pd.DataFrame().to_excel(company.name[:14]+'_arvore.xlsx')
    occ = OccurrenceType.objects.filter(company=company).values_list('occurrence_kind','name','form_fields__fields').exclude(occurrence_kind__isnull=True).order_by('occurrence_kind')
    campos=[]
    dados=[]
    for a in occ:
        try:
            natureza=get_connected_reference(company,
                                    "reporting",
                                    "occurrence_kind",
                                    a[0])
        except:
            continue
        else:
            campos = [field['displayName'] for field in a[2]]
            dados.append([natureza,a[1],campos])
    df2 = pd.DataFrame(data=dados,columns=['Natureza','Classe','Campos do Formulário'])
#         df2= df2.transpose()
    with pd.ExcelWriter(company.name[:14]+'_arvore.xlsx', mode='a',if_sheet_exists='replace') as writer:
        df2.to_excel(writer,sheet_name=company.name,na_rep="-")

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
df2

,Natureza,Classe,Campos do Formulário
0,Levantamento,Fiscalização,[ PLANEJAMENTO DIÁRIO DE SEGURANÇA DO TRABALHO...
1,Drenagem,Implantação de placa de regulamentação ou adve...,"[Origem, Código Interno, Código Kcor, Código A..."
2,Drenagem,Remoção de placa de solo,"[Origem, Código Interno, Código Kcor, Código A..."
3,Drenagem,Implantação de placa com chapa de aço galvaniz...,"[Origem, Código Interno, Código Kcor, Código A..."
4,Drenagem,Remoção de placa area,"[Origem, Código Interno, Código Kcor, Código A..."
...,...,...,...
79,"Cercas, alambrados e telamentos",Implantação (Cercas/Alambrados/Telamento),"[Origem, Código Interno, Código Kcor, Código A..."
80,Defensa Metálica,Implantação (Defensas metálicas),"[Origem, Código Interno, Código Kcor, Código A..."
81,Defensa Metálica,Alinhamento (Defensa Metálica),"[Origem, Código Interno, Código Kcor, Código A..."
82,Defensa Metálica,Reparo (Defensa Metálica),"[Origem, Código Interno, Código Kcor, Código A..."


# tipos de serviço exceção

In [20]:
unidades=[]
for company in tqdm(companies):
    if company.name not in unidades:
        unidades.append(company.name)
        pd.DataFrame().to_excel(company.name[:14]+'_arvore.xlsx')
    occ = OccurrenceType.objects.filter(company=company).values_list('occurrence_kind','name','form_fields__fields').exclude(occurrence_kind__isnull=True).order_by('occurrence_kind')
    
    dados=[]
    for a in occ:
        try:
            natureza=get_connected_reference(company,
                                    "reporting",
                                    "occurrence_kind",
                                    a[0])
        except:
            continue
        else:
            campos=[]
            for field in a[2]:
                if 'tipo de serviço' in field['displayName'].lower():
                    campos = [option['name'] for option in field['selectOptions']['options']]
            dados.append([natureza,a[1],campos])
    df2 = pd.DataFrame(data=dados,columns=['Natureza','Classe','Campos do Formulário'])
#         df2= df2.transpose()
    with pd.ExcelWriter(company.name[:14]+'_arvore.xlsx', mode='a',if_sheet_exists='replace') as writer:
        df2.to_excel(writer,sheet_name=company.name,na_rep="-")

  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
df2[df2['Classe'] == "Drenagem Profunda"]

,Natureza,Classe,Campos do Formulário
22,Drenagem,Drenagem Profunda,"[Enleivamento, Limpeza (Drenagem Profunda), Pi..."
